# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2a2a7ba14b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

In [31]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [66]:
# getting the value for each property 
queryString = """
SELECT DISTINCT ?p ?name ?value
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?value
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'family name'), ('value', 'James')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('value', '2006 FIBA World Championship')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for'), ('value', 'Laureus World Sports Award for Breakthrough of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('value', 'J. Walter Kennedy Citizenship Award')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth'), ('value', 'Akron')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('name', 'residence'), ('value', 'Akron')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for'), ('value', 'Best Championship Performance ESPY Award')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('value', 'Best Championship Performance ESPY Award')]
[('p', 'http://www.wikidata

In [7]:
# getting the value for each property 
queryString = """
SELECT DISTINCT ?obj ?value
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?value
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('value', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('value', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('value', 'basketball player')]
3


Final query for this task

In [9]:
#FINAL QUERY 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q3665646 ?p ?obj .
    # get the label
    ?p sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.w

## Task 2
Identify the BGP for basketball.

In [10]:
#instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q3665646 wdt:P31 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
1


In [12]:

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q3665646 wdt:P279 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('obj', 'http://www.wikidata.org/entity/Q2066131'), ('name', 'athlete')]
2


In [13]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q3665646 wdt:P361 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


In [14]:
#basketball 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q3665646 wdt:P641 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


Final query for this task

In [16]:
#FINAL QUERY 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q5372 ?p ?obj .
    # get the label
    ?p sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1807'), ('name', 'Great Aragonese En

## Task 3
Identify the BGP for basketball team.

In [16]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q3665646 wdt:P361 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


Final query for this task

In [19]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q13393265 ?p ?obj .
    # get the label
    ?p sc:name ?name.
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [21]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q36159 wdt:P54 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
3


In [22]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    wd:Q169138 wdt:P31 ?obj .
    # get the label
    ?obj sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


In [28]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?pValue ?obj ?name
WHERE {
    wd:Q169138 ?p ?obj .
    # get the label
    ?p sc:name ?pValue .
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pValue', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8624414'), ('name', 'Category:Miami Heat')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pValue', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q639928'), ('name', 'Southeast Division')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pValue', 'home venue'), ('obj', 'http://www.wikidata.org/entity/Q420129'), ('name', 'American Airlines Arena')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pValue', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pValue', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pValue', 'league'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('p', 'http://www.

In [2]:
# Los Angeles Lakers
queryString = """
SELECT DISTINCT ?p ?pValue ?obj ?name
WHERE {
    wd:Q121783 ?p ?obj .
    # get the label
    ?p sc:name ?pValue .
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pValue', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pValue', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pValue', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q65'), ('name', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pValue', 'league'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pValue', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q1781055'), ('name', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pValue', 'home venue'), ('obj', 'http://www.wikidata.org/entity/Q186080'), ('name', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/dir

In [3]:
# BGP for basketball
queryString = """
SELECT DISTINCT ?p ?pValue ?obj ?name
WHERE {
    wd:Q5372 ?p ?obj .
    # get the label
    ?p sc:name ?pValue .
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('pValue', 'uses'), ('obj', 'http://www.wikidata.org/entity/Q2341939'), ('name', 'basketball court')]
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('pValue', 'on focus list of Wikimedia project'), ('obj', 'http://www.wikidata.org/entity/Q15304953'), ('name', 'WikiProject African diaspora')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pValue', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pValue', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q216048'), ('name', 'team sport')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pValue', 'country of origin'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pValue', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q31629'), ('name', 'type of sport')]
[('p

In [6]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?country ?countryName
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 .
    ?teamIRI wdt:P17 ?obj .
    # get the label
    ?teamIRI sc:name ?team .
    ?country sc:name ?countryName .
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/P348'), ('countryName', 'software version identifier')]
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/P4669'), ('countryName', 'tabular software version')]
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/P747'), ('countryName', 'has edition or translation')]
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/Q1002728'), ('countryName', 'Fonsi Nieto')]
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/Q100326534'), ('countryName', "Al Stewart's albums in chronological order")]
[('team', "2013–14 Atlantic Coast Conference men's basketball season"), ('country', 'http://www.wikidata.org/entity/Q100327667

In [36]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?country ?countryName
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 .
    ?teamIRI wdt:P17 ?obj .
    # get the label
    ?teamIRI sc:name ?team .
    ?country sc:name ?countryName .
}
GROUP BY ?team
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error The estimated execution time -1567211520 (sec) exceeds the limit of 800 (sec).\n\nSPARQL query:\n\n##-2a2a7ba14b-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?team ?country ?countryName\nWHERE {\n    ?teamIRI wdt:P641 wd:Q5372 .\n    ?teamIRI wdt:P17 ?obj .\n    # get the label\n    ?teamIRI sc:name ?team .\n    ?country sc:name ?countryName .\n}\nGROUP BY ?team\nLIMIT 30\n'


In [12]:
# write your queries
queryString = """
SELECT DISTINCT ?teamIRI ?team ?country ?countryName
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 .
    ?teamIRI wdt:P17 ?country .
    # get the label
    ?teamIRI sc:name ?team .
    ?country sc:name ?countryName .
    #FILTER (REGEX(?countryName, "Italy"))
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('teamIRI', 'http://www.wikidata.org/entity/Q18286220'), ('team', 'Metros de Santiago'), ('country', 'http://www.wikidata.org/entity/Q786'), ('countryName', 'Dominican Republic')]
[('teamIRI', 'http://www.wikidata.org/entity/Q2394357'), ('team', '2005 FIBA Americas Championship'), ('country', 'http://www.wikidata.org/entity/Q786'), ('countryName', 'Dominican Republic')]
[('teamIRI', 'http://www.wikidata.org/entity/Q18203083'), ('team', 'Titanes del Distrito Nacional'), ('country', 'http://www.wikidata.org/entity/Q786'), ('countryName', 'Dominican Republic')]
[('teamIRI', 'http://www.wikidata.org/entity/Q303061'), ('team', "Mauritania men's national basketball team"), ('country', 'http://www.wikidata.org/entity/Q1025'), ('countryName', 'Mauritania')]
[('teamIRI', 'http://www.wikidata.org/entity/Q20982411'), ('team', "Ethiopia men's national under-16 basketball team"), ('country', 'http://www.wikidata.org/entity/Q115'), ('countryName', 'Ethiopia')]
[('teamIRI', 'http://www.wikid

In [18]:
# write your queries
queryString = """
SELECT DISTINCT ?country ?teamIRI ?team 
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 .
    ?teamIRI wdt:P17 ?country .
    # get the label
    ?teamIRI sc:name ?team .
    ?country sc:name ?countryName .
    FILTER (REGEX(?countryName, "Italy"))
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q17332997'), ('team', 'Palavesuvio')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q16531388'), ('team', 'Virtus Basket Ariano')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q16586535'), ('team', 'Pallacanestro Brindisi')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q17057058'), ('team', '1969–70 Pallacanestro Brindisi season')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q17154474'), ('team', '1964–65 Pallacanestro Brindisi season')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/entity/Q17622442'), ('team', '2013–14 Basket Ariano Irpino season')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('teamIRI', 'http://www.wikidata.org/ent

In [13]:
# 1968–69 Pallacanestro Cantù season
queryString = """
SELECT DISTINCT ?p ?pValue ?obj ?name
WHERE {
    wd:Q24066100 ?p ?obj .
    # get the label
    ?p sc:name ?pValue .
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pValue', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q24066097'), ('name', '1967–68 Pallacanestro Cantù season')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pValue', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q24066101'), ('name', '1969–70 Pallacanestro Cantù season')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pValue', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pValue', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pValue', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q26894053'), ('name', 'basketball team season')]
[('p', 'http://www.wikidata.org/prop/direct/P5138'), ('pValue', 'season of club or team'), ('obj', 'http://www.wikidata.org/entity/Q36311'), ('name', 'Pallacanestro Cantù')]
[('p', 'h

In [15]:
# Basket Livorno
queryString = """
SELECT ?p ?pValue ?obj ?name
WHERE {
    wd:Q654461 ?p ?obj .
    # get the label
    ?p sc:name ?pValue .
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pValue', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pValue', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pValue', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q6761'), ('name', 'Livorno')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pValue', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q6761'), ('name', 'Livorno')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pValue', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pValue', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q21971304'), ('name', 'Category:Basket Livorno')]
6


In [5]:
# Basket Livorno inception year
# bsketball teams dont have this property 
queryString = """
SELECT ?obj ?name
WHERE {
    wd:Q654461 wdt:P571 ?obj .
    # get the label
    ?obj sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# Basket Livorno inception year
# bsketball teams dont have this property 
queryString = """
SELECT ?obj
WHERE {
    wd:Q654461 wdt:P571 ?obj .
    # get the label
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('obj', '1947-01-01T00:00:00Z')]
1


In [8]:
queryString = """
SELECT DISTINCT ?teamIRI ?team ?inception
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 ;
             wdt:P17 wd:Q38 ;
             wdt:P571 ?inception .
    # get the label
    ?teamIRI sc:name ?team .
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('teamIRI', 'http://www.wikidata.org/entity/Q16586535'), ('team', 'Pallacanestro Brindisi'), ('inception', '1945-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q20994305'), ('team', 'A.S.D. Polisportiva Olonia'), ('inception', '1973-06-12T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q21001783'), ('team', 'Basket Ravenna Piero Manetti'), ('inception', '1984-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q3093425'), ('team', 'Libertas Trogylos Basket'), ('inception', '1970-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q3515600'), ('team', 'Taranto Cras Basket'), ('inception', '1961-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36231'), ('team', 'S.S. Felice Scandone'), ('inception', '1948-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36251'), ('team', 'Pallacanestro Biella'), ('inception', '1994-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36419'), ('team', 

In [13]:
queryString = """
SELECT DISTINCT ?teamIRI ?team ?inception
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 ;
             wdt:P17 wd:Q38 ;
             wdt:P571 ?inception .
    # get the label
    ?teamIRI sc:name ?team .
    FILTER(?inception < "19450101"^^xsd:date)
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('teamIRI', 'http://www.wikidata.org/entity/Q16586535'), ('team', 'Pallacanestro Brindisi'), ('inception', '1945-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q2887228'), ('team', 'Reyer Venezia Mestre Femminile'), ('inception', '1925-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q3891980'), ('team', 'Nuova Pallacanestro Pavia'), ('inception', '1943-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q734750'), ('team', "Italy men's national basketball team"), ('inception', '1932-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36551'), ('team', 'Pallacanestro Varese'), ('inception', '1945-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q1139737'), ('team', 'Fortitudo Pallacanestro Bologna'), ('inception', '1932-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36362'), ('team', 'Olimpia Milano'), ('inception', '1936-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36

Final query for this task

In [15]:
# write your final query
queryString = """
SELECT DISTINCT ?teamIRI ?team ?inception
WHERE {
    ?teamIRI wdt:P641 wd:Q5372 ;
             wdt:P17 wd:Q38 ;
             wdt:P571 ?inception .
    # get the label
    ?teamIRI sc:name ?team .
    FILTER(?inception < "19450101"^^xsd:date)
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('teamIRI', 'http://www.wikidata.org/entity/Q16586535'), ('team', 'Pallacanestro Brindisi'), ('inception', '1945-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q2887228'), ('team', 'Reyer Venezia Mestre Femminile'), ('inception', '1925-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q3891980'), ('team', 'Nuova Pallacanestro Pavia'), ('inception', '1943-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q734750'), ('team', "Italy men's national basketball team"), ('inception', '1932-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36551'), ('team', 'Pallacanestro Varese'), ('inception', '1945-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q1139737'), ('team', 'Fortitudo Pallacanestro Bologna'), ('inception', '1932-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36362'), ('team', 'Olimpia Milano'), ('inception', '1936-01-01T00:00:00Z')]
[('teamIRI', 'http://www.wikidata.org/entity/Q36

## Task 5
Identify the BGP for NBA finals

In [26]:
# awards that Lebron James received 
queryString = """
SELECT ?award ?awardName 
WHERE {
    wd:Q36159 wdt:P166 ?award .
    # get the label
    ?award sc:name ?awardName .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1501719'), ('awardName', 'J. Walter Kennedy Citizenship Award')]
[('award', 'http://www.wikidata.org/entity/Q4896605'), ('awardName', 'Best Championship Performance ESPY Award')]
[('award', 'http://www.wikidata.org/entity/Q4896598'), ('awardName', 'Best Breakthrough Athlete ESPY Award')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('awardName', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('awardName', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('awardName', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('awardName', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awardName', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q31391'), ('awardName', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('award', 'h

In [28]:
#NBA Rookie of the Year Award is inscance of
queryString = """
SELECT ?obj ?name 
WHERE {
    wd:Q644357 wdt:P31 ?obj .
    # get the label
    ?obj sc:name ?name .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
1


In [29]:
#NBA Rookie of the Year Award is subclass of
queryString = """
SELECT ?obj ?name 
WHERE {
    wd:Q644357 wdt:P279 ?obj .
    # get the label
    ?obj sc:name ?name .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [39]:
#subclasses of award 
queryString = """
SELECT ?s ?name 
WHERE {
    ?s wdt:P279 wd:Q618779 .
    # get the label
    ?s sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q911283'), ('name', 'brevet')]
[('s', 'http://www.wikidata.org/entity/Q18157145'), ('name', 'YouTube Creator Awards')]
[('s', 'http://www.wikidata.org/entity/Q1108877'), ('name', 'collar')]
[('s', 'http://www.wikidata.org/entity/Q20103352'), ('name', 'Premi Llibreter de narrativa')]
[('s', 'http://www.wikidata.org/entity/Q5557260'), ('name', 'Hijo Predilecto')]
[('s', 'http://www.wikidata.org/entity/Q738903'), ('name', 'Yellow jersey')]
[('s', 'http://www.wikidata.org/entity/Q740089'), ('name', 'Leo Awards')]
[('s', 'http://www.wikidata.org/entity/Q1046088'), ('name', 'hall of fame')]
[('s', 'http://www.wikidata.org/entity/Q2548939'), ('name', 'War Resisters League Peace Award')]
[('s', 'http://www.wikidata.org/entity/Q484963'), ('name', 'BMI Film & TV Awards')]
[('s', 'http://www.wikidata.org/entity/Q99864'), ('name', 'Freedom of the City')]
[('s', 'http://www.wikidata.org/entity/Q18328126'), ('name', 'video game award')]
[('s', 'http://w

In [40]:
queryString = """
SELECT ?s ?name 
WHERE {
    ?s wdt:P279 wd:Q618779 .
    wd:Q13393265 wdt:P166 ?s .
    # get the label
    ?s sc:name ?name .
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [32]:
# awards that BT Miami Heat received 
queryString = """
SELECT ?award ?awardName 
WHERE {
    wd:Q169138 wdt:P166 ?award .
    # get the label
    ?award sc:name ?awardName .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [41]:
queryString = """
SELECT DISTINCT ?p ?name ?value
WHERE {
   # bind something
   wd:Q618779 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?value
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P366'), ('name', 'use'), ('value', 'recognition')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('value', 'recognition')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('name', 'partially coincident with'), ('value', 'competition')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list'), ('value', 'list of lists of prizes, medals, and awards')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('value', 'distinction')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('value', 'New Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('value', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('value', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/pro

In [42]:
#Lebron-leage-NBA
queryString = """
SELECT DISTINCT ?obj ?name 
WHERE {
   # bind something
   wd:Q36159 wdt:P118 ?obj .
   # get the label
   ?obj sc:name ?name
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
1


In [4]:
#NBA 
queryString = """
SELECT DISTINCT ?p ?value ?obj ?name 
WHERE {
   # bind something
   wd:Q155223 ?p ?obj .
   # get the label
   ?p sc:name ?value .
   ?obj sc:name ?name
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('value', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q18536323'), ('name', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('value', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15986354'), ('name', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('value', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q238240'), ('name', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('value', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q2455323'), ('name', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('value', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q810343'), ('name', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('value', 'season ends'), ('obj', 'http://www.wikidata.org/entity/Q118'), ('name', 'April')]
[('p', 'http://www.wikidata.org/prop/di

In [47]:
#subject - sport - basketball
queryString = """
SELECT DISTINCT ?s ?value
WHERE {
   # bind something
   ?s wdt:P641 wd:Q5372 .
   # get the label
   ?s sc:name ?value .
   FILTER (REGEX(?value, 'NBA Finals'))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q24264147'), ('value', '2016 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q29096468'), ('value', '2017 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q1518653'), ('value', 'WNBA Finals Most Valuable Player Award')]
[('s', 'http://www.wikidata.org/entity/Q842375'), ('value', 'NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q4588662'), ('value', '1993 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q4598928'), ('value', '2001 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q739499'), ('value', 'Bill Russell NBA Finals Most Valuable Player Award')]
[('s', 'http://www.wikidata.org/entity/Q4615114'), ('value', '2009 WNBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q4627226'), ('value', '2012 WNBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q15304758'), ('value', '2013 WNBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q24886436'), ('value', 'NBA Finals television ratings')]
[('s', 'http://www.wikidata.o

Final query for this task

In [48]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q842375 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('name', 'official name')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('name', 'month of the year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), (

## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [50]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P54 ?team .
   # get the label
   ?team sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('team', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
3


In [19]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   wd:Q29545 wdt:P54 ?team .
   # get the label
   ?team sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team")]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors')]
6


In [17]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   wd:Q29545 wdt:P54 ?team .
   wd:Q36159 wdt:P54 ?team .
   # get the label
   ?team sc:name ?name.
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [21]:
#1995 NBA Finals
queryString = """
SELECT DISTINCT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q4590500 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?value.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q161345'), ('value', 'Houston Rockets')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('value', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('value', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q842375'), ('value', 'NBA Finals')]
4


In [24]:
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   wd:Q29545 wdt:P54 ?team .
   ?award wdt:P31 wd:Q842375 ;
          wdt:P1346 ?team .
   # get the label
   ?team sc:name ?name.
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors')]
2


In [25]:
queryString = """
SELECT DISTINCT ?team ?name (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?team wdt:P31 wd:Q13393265.
   ?award wdt:P31 wd:Q842375 ;
          wdt:P1346 ?team .
   # get the label
   ?team sc:name ?name.
}
GROUP BY ?team ?name
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q161345'), ('name', 'Houston Rockets'), ('victories', '2')]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors'), ('victories', '4')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('name', 'Philadelphia 76ers'), ('victories', '3')]
[('team', 'http://www.wikidata.org/entity/Q159893'), ('name', 'Atlanta Hawks'), ('victories', '1')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers'), ('victories', '16')]
[('team', 'http://www.wikidata.org/entity/Q2928124'), ('name', 'Baltimore Bullets'), ('victories', '1')]
[('team', 'http://www.wikidata.org/entity/Q159729'), ('name', 'San Antonio Spurs'), ('victories', '5')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat'), ('victories', '2')]
[('team', 'http://www.wikidata.org/entity/Q169661'), ('name', 'Detroit Pistons'), ('victories', '3')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('name', '

In [28]:
queryString = """
SELECT DISTINCT ?team ?name (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?team wdt:P31 wd:Q13393265.
   ?award wdt:P31 wd:Q842375 ;
          wdt:P1346 ?team .
   # get the label
   ?team sc:name ?name.
   wd:Q29545 wdt:P54 ?team .
   wd:Q36159 wdt:P54 ?team .
}
GROUP BY ?team ?name
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [44]:
queryString = """
SELECT DISTINCT ?team ?name (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?team wdt:P31 wd:Q13393265.
   ?award wdt:P31 wd:Q842375 ;
          wdt:P1346 ?team .
   # get the label
   ?team sc:name ?name.
   ?player wdt:P54 ?team ;
          wdt:P1559 ?playerName.
   FILTER(?playerName = "LeBron James" || ?playerName = "Kevin Durant")
}
GROUP BY ?team ?name
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [2]:
# write your final query
queryString = """
SELECT DISTINCT ?team ?name (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?team wdt:P31 wd:Q13393265.
   ?award wdt:P31 wd:Q842375 ;
          wdt:P1346 ?team .
   # get the label
   ?team sc:name ?name.
   ?player wdt:P54 ?team ;
          wdt:P1559 ?playerName.
   FILTER(?playerName = "LeBron James" || ?playerName = "Kevin Durant")
}
GROUP BY ?team ?name
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [48]:
# write your queries
queryString = """
SELECT DISTINCT ?team ?name ?playerName
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223 ;
         sc:name ?name.
   ?player wdt:P54 ?team ;
          sc:name ?playerName.
          
}
GROUP BY ?team ?name
LIMIT 5
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q163480'), ('name', 'Charlotte Hornets'), ('playerName', 'Khalid Reeves')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards'), ('playerName', 'Alonzo Gee')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards'), ('playerName', 'Chris Singleton')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards'), ('playerName', 'Eddie Miles')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards'), ('playerName', 'Horace Walker')]
5


In [50]:
# write your queries
queryString = """
SELECT DISTINCT ?playerName ?country ?countryName
WHERE {
   # bind something
   ?player wdt:P27 ?country ;
          sc:name ?playerName.
   ?country sc:name ?countryName .
          
}
GROUP BY ?team ?name
LIMIT 5
"""
print("Results")
x=run_query(queryString)

Results
[('playerName', 'Johann Christoph Fugger'), ('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('playerName', 'Chen Chongzhan'), ('country', 'http://www.wikidata.org/entity/Q9903'), ('countryName', 'Ming dynasty')]
[('playerName', 'Diego Arán'), ('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('playerName', 'Robert Currie'), ('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('playerName', 'Morten Stokstad'), ('country', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway')]
5


In [56]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?value 
WHERE {
   wd:Q183 ?p ?obj.
   ?p sc:name ?value .
   ?obj sc:name ?name .
}
GROUP BY ?team ?name
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('value', 'diplomatic relation')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('value', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('value', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('value', 'shares border with')]
[('p', 'http://www.wikidata.org/prop/direct/P832'), ('value', 'public holiday')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('value', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('value', 'language used')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('value', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('value', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('value', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('value', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('value',

In [58]:
#P27 country of citinzenship 
# P30 continent 
# write your queries
queryString = """
SELECT DISTINCT ?team ?name ?playerName
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223 ;
         sc:name ?name.
   ?player wdt:P54 ?team ;
          sc:name ?playerName;
          wdt:P27 ?country .
    ?country wdt:P30 ?continent.
          
}
GROUP BY ?team ?name
LIMIT 5
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q164177'), ('name', 'Phoenix Suns'), ('playerName', 'Jeff Cook')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('name', 'Philadelphia 76ers'), ('playerName', 'Thaddeus Young')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('name', 'Philadelphia 76ers'), ('playerName', 'Charles Jenkins')]
[('team', 'http://www.wikidata.org/entity/Q172339'), ('name', 'New Orleans Pelicans'), ('playerName', 'Mike James')]
[('team', 'http://www.wikidata.org/entity/Q172339'), ('name', 'New Orleans Pelicans'), ('playerName', 'Jeff Grayer')]
5


In [59]:
#P27 country of citinzenship 
# P30 continent 
# write your queries
queryString = """
SELECT DISTINCT ?continent (COUNT(?player) AS ?players)
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223 ;
         sc:name ?name.
   ?player wdt:P54 ?team ;
          sc:name ?playerName;
          wdt:P27 ?country .
    ?country wdt:P30 ?continent.
          
}
GROUP BY ?continent
LIMIT 5
"""
print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q48'), ('players', '178')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('players', '80')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('players', '45')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('players', '132')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('players', '796')]
5


Final query for this task

In [61]:
# write your final query
queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(?player) AS ?players)
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223 ;
         sc:name ?name.
   ?player wdt:P54 ?team ;
          sc:name ?playerName;
          wdt:P27 ?country .
    ?country wdt:P30 ?continent.
    ?continent sc:name ?continentName .
}
GROUP BY ?continent ?continentName
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '80')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '9381')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '9550')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '796')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '45')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '178')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '132')]
7


## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

In [5]:
# write your queries
# awards that Lebron James received 
queryString = """
SELECT ?award ?awardName 
WHERE {
    wd:Q36159 wdt:P166 ?award .
    # get the label
    ?award sc:name ?awardName .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1501719'), ('awardName', 'J. Walter Kennedy Citizenship Award')]
[('award', 'http://www.wikidata.org/entity/Q4896605'), ('awardName', 'Best Championship Performance ESPY Award')]
[('award', 'http://www.wikidata.org/entity/Q4896598'), ('awardName', 'Best Breakthrough Athlete ESPY Award')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('awardName', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('awardName', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('awardName', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('awardName', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awardName', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q31391'), ('awardName', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('award', 'h

Final query for this task

In [7]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q222047 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('name', 'name in kana')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [10]:
# write your queries
# select players that won the NBA most valuable palyer award
queryString = """
SELECT DISTINCT ?playerIRI ?label
WHERE {
   # bind something
   ?playerIRI wdt:P166 wd:Q222047 .
   # get the label
   ?playerIRI sc:name ?label.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('playerIRI', 'http://www.wikidata.org/entity/Q179051'), ('label', 'Kareem Abdul-Jabbar')]
[('playerIRI', 'http://www.wikidata.org/entity/Q134183'), ('label', 'Magic Johnson')]
[('playerIRI', 'http://www.wikidata.org/entity/Q169452'), ('label', "Shaquille O'Neal")]
[('playerIRI', 'http://www.wikidata.org/entity/Q17717'), ('label', 'Oscar Robertson')]
[('playerIRI', 'http://www.wikidata.org/entity/Q182455'), ('label', 'Wilt Chamberlain')]
[('playerIRI', 'http://www.wikidata.org/entity/Q189240'), ('label', 'Kevin Garnett')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190094'), ('label', 'Allen Iverson')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190152'), ('label', 'Larry Bird')]
[('playerIRI', 'http://www.wikidata.org/entity/Q192707'), ('label', 'Charles Barkley')]
[('playerIRI', 'http://www.wikidata.org/entity/Q193020'), ('label', 'Tim Duncan')]
[('playerIRI', 'http://www.wikidata.org/entity/Q196185'), ('label', 'Russell Westbrook')]
[('playerIRI', 'http://www.wikida

In [12]:
queryString = """
SELECT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q222047 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?value .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q179051'), ('value', 'Kareem Abdul-Jabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('value', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('value', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q134183'), ('value', 'Magic Johnson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q17717'), ('value', 'Oscar Robertson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q182455'), ('value', 'Wilt Chamberlain')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')

In [16]:

queryString = """
SELECT ?winnerName (COUNT(?winner) AS ?victories)
WHERE {
   # bind something
   wd:Q222047 wdt:P1346 ?winner .
   # get the label
   ?winner sc:name ?winnerName .
}
GROUP BY ?winnerName
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Magic Johnson'), ('victories', '1')]
[('winnerName', 'Russell Westbrook'), ('victories', '1')]
[('winnerName', 'Bill Russell'), ('victories', '1')]
[('winnerName', 'Dirk Nowitzki'), ('victories', '1')]
[('winnerName', 'Hakeem Olajuwon'), ('victories', '1')]
[('winnerName', 'Bob McAdoo'), ('victories', '1')]
[('winnerName', 'Derrick Rose'), ('victories', '1')]
[('winnerName', 'Karl Malone'), ('victories', '1')]
[('winnerName', 'Kevin Durant'), ('victories', '1')]
[('winnerName', 'Zach Brown'), ('victories', '1')]
[('winnerName', 'Kareem Abdul-Jabbar'), ('victories', '1')]
[('winnerName', 'Charles Barkley'), ('victories', '1')]
[('winnerName', 'Julius Erving'), ('victories', '1')]
[('winnerName', 'Stephen Curry'), ('victories', '1')]
[('winnerName', 'Kevin Garnett'), ('victories', '1')]
[('winnerName', 'David Robinson'), ('victories', '1')]
[('winnerName', 'James Harden'), ('victories', '1')]
[('winnerName', 'Wilt Chamberlain'), ('victories', '1')]
[('winnerName'

In [18]:
# count the awards of each player 
queryString = """
SELECT DISTINCT ?playerIRI ?label (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?playerIRI wdt:P166 ?award .

   # get the label
   ?playerIRI sc:name ?label.
}
GROUP BY ?playerIRI ?label
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('playerIRI', 'http://www.wikidata.org/entity/Q2173601'), ('label', 'Rudolf Pressberger'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q7343332'), ('label', 'Robert D. Maurer'), ('victories', '8')]
[('playerIRI', 'http://www.wikidata.org/entity/Q10287419'), ('label', 'Vai-Vai'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q28500856'), ('label', 'Alexander Chentsov'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q1657795'), ('label', 'Igor Kon'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q91898858'), ('label', 'Raymonde Bestaux'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q2959669'), ('label', 'Charles Louis Dinet'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q4485007'), ('label', 'Leonid Firsov'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q15969040'), ('label', 'Pierre François Dumont'), ('victories', '1')]
[('pl

In [18]:
# count the awards of each player 
queryString = """
SELECT DISTINCT ?playerIRI ?label (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?playerIRI wdt:P166 ?award .
   ?award wdt:P31 wd:Q652965 .
   
   FILTER(REGEX(?awardName, "NBA Most Valuable Player Award"))
   # get the label
   ?playerIRI sc:name ?label.
   ?award sc:name ?awardName .
}
GROUP BY ?playerIRI ?label 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('playerIRI', 'http://www.wikidata.org/entity/Q264393'), ('label', 'Dave Cowens'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q380515'), ('label', 'Willis Reed'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q192955'), ('label', 'Steve Nash'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q136678'), ('label', 'James Harden'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q433521'), ('label', 'Sylvia Fowles'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q179051'), ('label', 'Kareem Abdul-Jabbar'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q25369'), ('label', 'Kobe Bryant'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q36159'), ('label', 'LeBron James'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q41421'), ('label', 'Michael Jordan'), ('victories', '1')]
[('playerIRI', 'http://www.wikidata.org/entit

In [14]:
# count the awards of each player 
queryString = """
SELECT ?playerIRI ?label (COUNT(?award) AS ?victories)
WHERE {
   # bind something
   ?award wdt:P31 wd:Q652965 .
   FILTER (REGEX(?awardName, "NBA Most Valuable Player Award"))

   {
       SELECT ?playerIRI ?label
       WHERE {
           ?playerIRI wdt:P166 ?award .
       }
   }

   # get the label
   ?playerIRI sc:name ?label.
}
GROUP BY ?playerIRI ?label
LIMIT 4
"""

print("Results")
x=run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error The estimated execution time 1073741824 (sec) exceeds the limit of 800 (sec).\n\nSPARQL query:\n\n##-2a2a7ba14b-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT ?playerIRI ?label (COUNT(?award) AS ?victories)\nWHERE {\n   # bind something\n   ?award wdt:P31 wd:Q652965 .\n   FILTER (REGEX(?awardName, "NBA Most Valuable Player Award"))\n\n   {\n       SELECT ?playerIRI ?label\n       WHERE {\n           ?playerIRI wdt:P166 ?award .\n       }\n   }\n\n   # get the label\n   ?playerIRI sc:name ?label.\n}\nGROUP BY ?playerIRI ?label\nLIMIT 4\n'


In [64]:
#wd:Q222047 
queryString = """
SELECT DISTINCT ?playerIRI ?label (COUNT(?award) AS ?vicrories)
WHERE {
   # bind something
   ?playerIRI wdt:P166 ?award ;
              sc:name ?label.
   ?award sc:name ?awardName .
   FILTER(REGEX(?awardName, "NBA Most Valuable Player Award"))
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('playerIRI', 'http://www.wikidata.org/entity/Q169452'), ('label', "Shaquille O'Neal"), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q199896'), ('label', 'Karl Malone'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q41421'), ('label', 'Michael Jordan'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190094'), ('label', 'Allen Iverson'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q193020'), ('label', 'Tim Duncan'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q134183'), ('label', 'Magic Johnson'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190152'), ('label', 'Larry Bird'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q29545'), ('label', 'Kevin Durant'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q192955'), ('label', 'Steve Nash'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q

Final query for this task

In [65]:
# write your final query
queryString = """
SELECT DISTINCT ?playerIRI ?label (COUNT(?award) AS ?vicrories)
WHERE {
   # bind something
   ?playerIRI wdt:P166 ?award ;
              sc:name ?label.
   ?award sc:name ?awardName .
   FILTER(REGEX(?awardName, "NBA Most Valuable Player Award"))
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('playerIRI', 'http://www.wikidata.org/entity/Q169452'), ('label', "Shaquille O'Neal"), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q199896'), ('label', 'Karl Malone'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q41421'), ('label', 'Michael Jordan'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190094'), ('label', 'Allen Iverson'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q193020'), ('label', 'Tim Duncan'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q134183'), ('label', 'Magic Johnson'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q190152'), ('label', 'Larry Bird'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q29545'), ('label', 'Kevin Durant'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q192955'), ('label', 'Steve Nash'), ('vicrories', '1')]
[('playerIRI', 'http://www.wikidata.org/entity/Q